In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, DefaultDataCollator
from datasets import Dataset
from trl import SFTConfig, SFTTrainer

In [2]:
retain_train_df = pd.read_parquet(r'C:\Users\FLopezP\Documents\GitHub\Unlearning-LLM\data\retain_train-00000-of-00001.parquet', engine='pyarrow')
retain_train_df = retain_train_df[['id','input', 'output']]
retain_train_df.head(10)

,id,input,output
0,6adbf83c-5071-4979-bedb-e5184b15650bsc1,"Fredericka Amber was born on December 21, 1969...",number is 889-867-1855. She can be reached at ...
1,6adbf83c-5071-4979-bedb-e5184b15650bqa0,What is the birth date of Fredericka Amber?,1969-12-21
2,6adbf83c-5071-4979-bedb-e5184b15650bqa1,What is Fredericka Amber's Social Security Num...,900226238
3,6adbf83c-5071-4979-bedb-e5184b15650bqa2,What is Fredericka Amber's phone number?,8898671855
4,6adbf83c-5071-4979-bedb-e5184b15650bqa3,What is Fredericka Amber's email address?,fredericka_amber@me.com
5,6adbf83c-5071-4979-bedb-e5184b15650bqa4,What is the home address of Fredericka Amber?,"5611 North 61st Avenue, Louisville, KY, 40258."
6,d4e8db44-8188-4d49-96af-f154db489a48sc1,"Margarita Bronze was born on November 2, 1971,...",reached via phone at 7396451535 and email at [...
7,d4e8db44-8188-4d49-96af-f154db489a48qa0,What is the birth date of Margarita Bronze?,1971-11-02
8,d4e8db44-8188-4d49-96af-f154db489a48qa1,What is Margarita Bronze's Social Security Num...,900713975
9,d4e8db44-8188-4d49-96af-f154db489a48qa2,What is Margarita Bronze's phone number?,7396451535


In [3]:
def concat_input_output(input, output):
  """It concatenates the input and the LLM output"""
  text = []
  for i,o in zip(input, output):
    text.append(f'{i}\n  {o}')
  return text

retain_train_df['text'] = concat_input_output(retain_train_df.input.values, retain_train_df.output.values)

In [4]:
olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf")
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
print(olmo)

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
    (rotary_emb): OlmoRotaryEmbedding()
  )
  (lm_head): 

In [6]:
df_aux = pd.DataFrame(data = retain_train_df['text']) 
dataset1 = Dataset.from_pandas(df_aux)

train_args = TrainingArguments(
    output_dir = r"C:\Users\FLopezP\Desktop\GIL\Unlearning",
    eval_strategy = "no",
    learning_rate = "1e-3",
    weight_decay = 0.01,
    remove_unused_columns = True,
    push_to_hub = False,
)

sft_config = SFTConfig(
    dataset_text_field="text",
    packing = True,
    output_dir = r"C:\Users\FLopezP\Desktop\GIL\Unlearning",
)

In [7]:
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)
dc_aux = DefaultDataCollator()

In [8]:
dataset1

Dataset({
    features: ['text'],
    num_rows: 1136
})

In [9]:
trainer = SFTTrainer(
    model = olmo, 
    args = sft_config, 
    data_collator = dc_aux,
    train_dataset = dataset1,
    tokenizer = tokenizer,
)

C:\Users\FLopezP\AppData\Local\Temp\ipykernel_13576\1642129802.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


In [13]:
torch.cuda.empty_cache()

**Problema de ejecución del trainer**

1. Errores de memoria de CUDA
2. Formato de texto no es compatible con el Trainer normal, se opta por usar SFTTrainer

In [14]:
%%time

trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 22.41 GiB is allocated by PyTorch, and 12.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)